## Workshop #3: Scoring

In [1]:
#initialization
import os
from pyrosetta import *
from pyrosetta.teaching import *
from pyrosetta.toolbox import pose_from_rcsb
from pyrosetta import PyMOLMover
init()

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.12+release.5ecebca5ecebcadacdf48ede3c1981444601bf1cd47ce0d 2018-03-23T13:02:49] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r174 2018.12+release.5ecebca 5ecebcadacdf48ede3c1981444601bf1cd47ce0d http://www.pyrosetta.org 2018-03-23T13:02:49
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-74376114 seed_offset=0 real_seed=-74376114
core.init.random: RandomGenerator:i

In [2]:
#getting laptop's ip address (must do "export my_ip = "10.1.14.176"" in terminal before opening Jupyter)
my_ip = os.environ["my_ip"]
#obtaining a protein
ras = pose_from_rcsb("6Q21")
#defining and printing the full-atom score function
scorefxn = get_fa_scorefxn()
print scorefxn
#defining a new scorefunction with only attractive and repulsive VDW energies
scorefxn2 = ScoreFunction()
scorefxn2.set_weight(fa_atr, 1.0)
scorefxn2.set_weight(fa_rep, 1.0)
#showing the score and score weights
print "Ras Score", scorefxn(ras) 
scorefxn.show(ras)
#scoring energies for residue 24 on ras
print "For residue 24:", ras.energies().show(24)

The file 6Q21.pdb already exists; this file will be overwritten.
PDB 6Q21 successfully loaded from the RCSB into 6Q21.pdb.
if the file 6Q21.clean.pdb already exists, it will be overwritten
PDB 6Q21.pdb successfully cleaned, non-ATOM lines removed
clean data written to 6Q21.clean.pdb
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 604 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.704172 seconds.
core.import_pose.import_pose: File '6Q21.clean.pdb' automatically determined to be of type PDB
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.da

core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.172413 seconds to load from binary
 3776.41205299
core.scoring: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000   -4240.991   -4240.991
 fa_rep                       0.550    4336.451    2385.048
 fa_sol                       1.000    2798.424    2798.424
 fa_intra_rep                 0.005    3034.836      15.174
 fa_intra_sol_xover4          1.000     180.944     180.944
 lk_ball_wtd                  1.000     -56.460     -56.460
 fa_elec                      1.000    -933.702    -933.702
 pro_close                   

In [ ]:
#getting hbonds and displaying them
from pyrosetta.toolbox import get_hbonds
hbond_set = get_hbonds(ras)
hbond_set.show(ras)
hbond_set.show(ras, 24)
print hbond_set

In [ ]:
#import cetuximab
pose = pose_from_rcsb("1YY9")
print "Pose number for resi 102 chain A:", pose.pdb_info().pdb2pose('A', 102)
print pose.pdb_info().pdb2pose('D', 102)
print pose.pdb_info().pdb2pose('A', 408)
rsd1 = pose.residue(926)
rsd2 = pose.residue(407)

In [ ]:
#energies
emap = EMapVector()
scorefxn.eval_ci_2b(rsd1, rsd2, pose,emap)
print "fa_atr:", emap[fa_atr]
print "fa_rep:", emap[fa_rep]
print "fa_sol:", emap[fa_sol]

In [ ]:
#show protein in PyMOL
pymol = PyMOLMover(my_ip, 65000)
pymol.apply(pose)

In [ ]:
#all energies 
pymol.send_energy(pose)

In [ ]:
#attractive VDW energies
pymol.send_energy(pose, fa_atr)

In [ ]:
#solvation energies
pymol.send_energy(pose, fa_sol)

In [ ]:
pymol.update_energy(True)
pymol.energy_type(fa_atr)
pymol.apply(pose)

In [ ]:
#label every hbond in pymol
pymol.send_hbonds(pose)

1.Interface energy. Write a program that can calculate the binding energy of EGFR to
cetuximab. You will need to make separate PDB files for the antigen, antibody, and
complex. In PyMOL, select one of these peptides, then use File→Save	Molecule.
Use the following formula for binding energy:
Ebinding = Ecomplex – Eantibody – Eantigen

In [ ]:
#showing both proteins in cetuximab complex (1YY9) and calculating binding energy
fab = pose_from_pdb("FabC225.pdb")
pymol.apply(fab)
sEGFR = pose_from_pdb("SEGFR.pdb")
pymol.apply(sEGFR)
E_complex = scorefxn(pose)
E_fab = scorefxn(fab)
E_sEGFR = scorefxn(sEGFR)
print "Cetuximab Score", E_complex 
print "Fab Score", E_fab
print "sEGFR Score", E_sEGFR
E_binding = E_complex - E_fab - E_sEGFR
print "E_binding", E_binding


In [ ]:
print pose.total_residue()

In [ ]:
import pandas as pd
import numpy as np
from pyrosetta.toolbox import get_secstruct
for x in xrange(1, 1044):
    N = AtomID(1, x)
    C = AtomID(2, x)
    print pose.conformation().bond_length(N, C)
dframe.to_csv('N_C_bondlengths_1YY9.csv')
   

In [ ]:
from pandas import DataFrame
for x in xrange(1, 1044):
    N = AtomID(1, x)
    C = AtomID(2, x)
    a = pose.conformation().bond_length(N, C)
    

In [ ]:
AtomID(1,slice(None))

In [ ]:
conf = pose.conformation()


In [ ]:
atree = conf.get_residue_mask

In [ ]:
# this pulls the first residue, and inspects it to find the documentation of all the methods
res = [j for j in pose.residues][2]
method_info = [(j,res.__getattribute__(j).__doc__) for j in dir(res) if inspect.ismethod(res.__getattribute__(j))]
for j,k in method_info:
    print j,'-'*20
    print k

In [ ]:
#print res.icoor()
#print res.chain()
pose.

In [ ]:
from pandas import DataFrame
df = DataFrame([[j_num+1,(j.atom('N').xyz()-j.atom('C').xyz()).norm()] for j_num,j in enumerate(pose.residues)])
df.columns = ['resi', 'bond length']
df.to_csv('1YY9_bondlengths.csv')
print df


In [ ]:
pwd

In [ ]:
import inspect
print inspect.getsource(pose.residues[1].__str__)